In [1]:
from astropy.stats import mad_std

from ffs import FFS

from astropy.io import fits
import os
import numpy as np

path = "/work/vela/oca/fits/zb08/raw"
#path = "/work/vela/oca/fits/zb08/processed-ofp/science"
noce = [d for d in os.listdir(path) if (os.path.isdir(path+"/"+d) and not os.path.islink(path+"/"+d) )]

noce = sorted(noce)

print(noce)

['0088', '0091', '0095', '0096', '0098', '0099', '0100', '0101', '0102', '0103', '0104', '0105', '0106', '0108', '0112', '0113', '0114', '0116', '0117', '0119', '0154', '0156', '0157', '0161', '0163', '0164', '0165', '0166', '0167', '0168', '0169', '0170', '0171', '0172', '0173', '0174', '0175', '0176', '0177', '0178', '0179', '0180', '0181', '0182', '0183', '0184', '0185', '0186', '0187', '0188', '0189', '0190', '0191', '0192', '0193', '0194', '0195', '0196', '0197', '0198', '0199', '0200', '0201', '0202', '0203', '0204', '0205', '0206', '0208', '0209', '0210', '0211', '0214', '0215', '0216', '0217', '0219', '0220', '0221', '0222', '0223', '0224', '0225', '0226', '0227', '0228', '0229', '0231', '0232', '0233', '0234', '0235', '0236', '0237', '0238', '0239', '0240', '0241', '0242', '0243', '0248', '0249', '0250', '0251', '0252', '0253', '0254', '0255', '0256', '0257', '0258', '0260', '0261', '0262', '0263', '0264', '0265', '0266', '0267', '0268', '0269', '0270', '0271', '0272', '0273',

In [2]:
fits_list = []
night = "1024"
for f in os.listdir(path+"/"+night):
    if os.path.isfile(path+"/"+night+"/"+f):
        fits_list.append(f)

print(len(fits_list))

1077


In [ ]:
for f in fits_list:
    hdul = fits.open(path+"/"+night+"/"+f)
    data = hdul[0].data
    hdr = hdul[0].header
    hdul.close()

    if hdr["IMAGETYP"] == "science":
        ffs = FFS(data)
        ffs.mk_stats()
        ffs.find_stars(threshold=5,kernel_size=7,fwhm=3)
        fwhm_x,fwhm_y = ffs.fwhm()
        ffs.sky_gradient()
        maska = data > np.median(data) + 3 * mad_std(data)
        line_val, line_tht, line_r = ffs.hough_transform(maska)


        #print(f,hdr["OBSTYPE"],hdr["IMAGETYP"],hdr["OBJECT"],ffs.stats)
        #print(hdr["IMAGETYP"],hdr["OBJECT"],ffs.stats["median"])
        line = 0
        if len(line_val) > 0:
            line = line_val[0]
        print(f,hdr["IMAGETYP"], hdr["FILTER"], hdr["OBJECT"], len(ffs.coo), ffs.fwhm_x, ffs.fwhm_y, ffs.max_amplitude, ffs.frame_gradient, line)

zb08c_1024_71826.fits science i HS_Car 7914 1.35625 1.4777777777777779 1.4235224978594374 1.4235224978594374 227.0
zb08c_1024_57282.fits science g BK_Tuc 31469 1.2303571428571427 1.4150300054555374 0.11081455133773943 0.11081455133773943 218.0
zb08c_1024_69891.fits science V RY_Col 5108 1.2861607142857143 1.3437098844672657 3.410605131648481e-13 3.410605131648481e-13 107.0
zb08c_1024_83688.fits science i ASAS_J110522-2641.0 20654 1.266257750973333 1.3901564451256103 3.030193591920522 3.030193591920522 147.0
zb08c_1024_59088.fits science V RX_Cet 3660 1.275797373358349 1.3166780665754674 0.1859382238969829 0.1812504740020131 None
zb08c_1024_71385.fits science Ic CN_Car 23170 1.2390988314517726 1.443050193050193 1.0603931120509174 1.0370143481579248 163.0
zb08c_1024_57001.fits science z RV_Phe 1161 1.4507246376811596 1.5382113821138212 15.68912168397992 13.508510216309162 143.0
zb08c_1024_78242.fits science z CD_Vel 2947 3.3172158869833286 3.5061440137149917 16.43286439996649 14.55111327